In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 정규화

# 라벨을 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [2]:
import optuna
from tensorflow.keras.optimizers import Adam

def objective(trial):
    # 하이퍼파라미터 정의
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    num_units = trial.suggest_int("num_units", 32, 128, step=32)

    # 모델 생성
    model = models.Sequential([
        layers.Conv2D(num_units, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(num_units, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # 모델 훈련
    model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=0)
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best hyperparameters:", study.best_params)


[I 2024-12-10 15:18:21,983] A new study created in memory with name: no-name-47d28fbf-5ccc-49e7-b959-b5850b85a0eb
c:\Users\envym\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'


[I 2024-12-10 15:21:10,041] Trial 0 finished with value: 0.6190999746322632 and parameters: {'learning_rate': 0.003746508630400722, 'num_units': 96}. Best is trial 0 with value: 0.6190999746322632.
[I 2024-12-10 15:21:42,752] Trial 1 finished with value: 0.49540001153945923 and parameters: {'learning_rate': 0.009009532019570289, 'num_units': 32}. Best is trial 0 with value: 0.6190999746322632.
[I 2024-12-10 15:24:43,565] Trial 2 finished with value: 0.5637999773025513 and parameters: {'learning_rate': 0.00010267950148422044, 'num_units': 96}. Best is trial 0 with value: 0.6190999746322632.
[I 2024-12-10 15:26:15,966] Trial 3 finished with value: 0.4925999939441681 and parameters: {'learning_rate': 0.0054753147627106735, 'num_units': 64}. Best is trial 0 with value: 0.6190999746322632.
[I 2024-12-10 15:30:09,543] Trial 4 finished with value: 0.6299999952316284 and parameters: {'learning_rate': 0.000390654890583853, 'num_units': 128}. Best is trial 4 with value: 0.6299999952316284.
[I 20

Best hyperparameters: {'learning_rate': 0.0009403547801367684, 'num_units': 64}


In [4]:
# 최적화된 파라미터 사용
best_params = study.best_params
final_model = models.Sequential([
    layers.Conv2D(best_params["num_units"], (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(best_params["num_units"], activation='relu'),
    layers.Dense(10, activation='softmax')
])

final_model.compile(optimizer=Adam(learning_rate=best_params["learning_rate"]),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# 모델 훈련
final_model.fit(x_train, y_train, epochs=10, batch_size=64)

# 모델 저장
final_model.export("saved_model/1")


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.3257 - loss: 1.8326
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.5369 - loss: 1.2989
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.5892 - loss: 1.1727
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.6150 - loss: 1.0971
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.6351 - loss: 1.0419
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.6555 - loss: 0.9900
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.6675 - loss: 0.9510
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.6802 - loss: 0.9196
Epoch 9/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.6920 - loss: 0.8827
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.7028 - loss: 0.8560
INFO:tensorflow:Assets written to: saved_model/1\assets


INFO:tensorflow:Assets written to: saved_model/1\assets


Saved artifact at 'saved_model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_66')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1851369128976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1851369137040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1851369138576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1851369129744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1851369131664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1851369142608: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [8]:
import requests
import numpy as np

# CIFAR-10 이미지 샘플 (32x32x3 크기의 더미 데이터 생성)
image = np.random.rand(32, 32, 3).tolist()

# REST API 요청
url = "http://localhost:8501/v1/models/cifar10_model:predict"
data = {"instances": [image]}
response = requests.post(url, json=data)

# 결과 출력
print("Predictions:", response.json())


Predictions: {'predictions': [[4.74354283e-06, 2.28325662e-05, 0.000140078919, 3.36624112e-07, 9.56019067e-05, 8.58904059e-09, 0.999700427, 1.68558714e-07, 3.39692585e-09, 3.58887191e-05]]}


In [9]:
from tensorflow.keras.datasets import cifar10

# 데이터 로드
(x_train, _), _ = cifar10.load_data()

# 첫 번째 샘플을 준비
data = x_train[:1].astype("float32") / 255.0  # 정규화

# REST API 요청
response = requests.post(url, json={"instances": data.tolist()})
print("Predictions:", response.json())


Predictions: {'predictions': [[8.44800525e-05, 2.74479044e-05, 0.0160447545, 0.215927765, 0.0262216274, 0.192304417, 0.53503114, 0.014258706, 2.84754551e-05, 7.11921311e-05]]}


In [11]:
import numpy as np
import requests

# [32x32x3] 크기의 샘플 데이터 생성
data = np.random.rand(1, 32, 32, 3).tolist()

# REST API 요청
url = "http://localhost:8501/v1/models/cifar10_model:predict"
response = requests.post(url, json={"instances": data})

# 결과 출력
print("Predictions:", response.json())


Predictions: {'predictions': [[1.46709272e-05, 0.00100184383, 6.02009568e-05, 1.03881557e-05, 0.000546160154, 4.389384e-07, 0.998273134, 3.08143062e-05, 4.04766958e-08, 6.21754662e-05]]}


In [12]:
import tensorflow as tf

# SavedModel 경로
saved_model_dir = "saved_model/1"

# TFLite 변환기
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# TFLite 모델 저장
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite 모델이 'model.tflite'로 저장되었습니다.")


TFLite 모델이 'model.tflite'로 저장되었습니다.
